# Weather Data - Data Cleaning and Preprocessing
## Data Science - CS 334
---------------------------------------------------------

### Dataset
- The weather dataset is a messy, real-world dataset containing an entire year’s worth of weather data from Boston, USA. 
- This dataframe has a lot of problems regarding the quality of the data. 
- The goal for this part of the assignment is to clean the data so it can be later used for insightful analysis.

### Importing Libraries
-----------------------------

In [102]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics as sts
sns.set()

In [103]:
#Loading data
df = pd.read_csv('weather_start.csv')
print(df.shape)
df.head()

(366, 23)


,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,'6','Rain','46','29','74','30.45','64','10','22','63',...,'13','40','26','52','30.01','39','10','0.01','268',Year: 2014 Month: 12 Day: 1
1,'7','Rain-Snow','40','29','92','30.71','42','10','24','72',...,'15','27','17','51','30.4','33','2','0.10','62',Year: 2014 Month: 12 Day: 2
2,'8','Rain','49','38','100','30.4','51','10','29','79',...,'12','42','24','57','29.87','37','1','0.44','254',Year: 2014 Month: 12 Day: 3
3,'3',NaN,'24','33','69','30.56','43','10','25','54',...,'12','21','13','39','30.09','30','10','0.00','292',Year: 2014 Month: 12 Day: 4
4,'5','Rain','37','26','85','30.68','42','10','22','66',...,'10','25','12','47','30.45','26','5','0.11','61',Year: 2014 Month: 12 Day: 5


<div style="color: pink; 
            background-color: #b73239;
            border-radius: 5px;
            padding: 5px 5px 5px 5px;
            display: inline-block">
    If you get a `SettingWithCopyWarning:` when using pandas, you can almost always ignore it. 
</div>

## Cleaning Part I: Data Formats and Types

--------------------------------------------------

- We can see the `Date` column has the information in a weird text-based style. So the first order of business is to parse the `Date` column and obtain separate year, month and day columns as integer values.You will use regex for this.Then you will combine these three columns using `pd.to_datetime` to create a datetime column which is a special type in pandas. It gives you some amazing funcitonality.
- Then you have individual weather related measurements (`Max.TemperatureF`, `Max.Dew.PointF` etc) but the values of these columns have quotes around them ('') so these need to be cleaned.
- For the weather related columns you need to fix the types. Pandas documentation is your friend so use it!
- You should also keep an eye out for inconsistent encodings.  Everything is a string, that means encoding issues can be present. These are more common in categorical variables like a column describing gender has both full words (female, male) and letters (f, m). You will need to properly standardise this issue.
- Everything is a string. This is a sign of a long day (maybe not?) of data cleaning ahead. Good Luck!

In [104]:
a= pd.read_pickle("weather_clean.pkl")
a

,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,6.0,Rain,46.0,29.0,74.0,30.45,64.0,10.0,22.0,63.0,...,13.0,40.0,26.0,52.0,30.01,39.0,10.0,0.01,268.0,2014-12-01
1,7.0,Rain-Snow,40.0,29.0,92.0,30.71,42.0,10.0,24.0,72.0,...,15.0,27.0,17.0,51.0,30.40,33.0,2.0,0.10,62.0,2014-12-02
2,8.0,Rain,49.0,38.0,100.0,30.40,51.0,10.0,29.0,79.0,...,12.0,42.0,24.0,57.0,29.87,37.0,1.0,0.44,254.0,2014-12-03
3,3.0,None,24.0,33.0,69.0,30.56,43.0,10.0,25.0,54.0,...,12.0,21.0,13.0,39.0,30.09,30.0,10.0,0.00,292.0,2014-12-04
4,5.0,Rain,37.0,26.0,85.0,30.68,42.0,10.0,22.0,66.0,...,10.0,25.0,12.0,47.0,30.45,26.0,5.0,0.11,61.0,2014-12-05
5,8.0,Rain,45.0,25.0,100.0,30.42,45.0,10.0,22.0,93.0,...,8.0,40.0,36.0,85.0,30.16,38.0,0.0,1.09,313.0,2014-12-06
6,6.0,Rain,36.0,32.0,92.0,30.69,38.0,10.0,25.0,61.0,...,15.0,20.0,-3.0,29.0,30.24,21.0,5.0,0.13,350.0,2014-12-07
7,8.0,Snow,28.0,28.0,92.0,30.77,29.0,10.0,21.0,70.0,...,13.0,16.0,3.0,47.0,30.51,18.0,2.0,0.03,354.0,2014-12-08
8,8.0,Rain,49.0,52.0,100.0,30.51,49.0,10.0,38.0,93.0,...,20.0,41.0,28.0,86.0,29.49,29.0,1.0,2.90,38.0,2014-12-09
9,8.0,Rain,45.0,29.0,100.0,29.58,48.0,10.0,23.0,95.0,...,13.0,39.0,37.0,89.0,29.43,38.0,1.0,0.28,357.0,2014-12-10


In [105]:
dt= df["Date"].str.split(expand=True)
df['Date']= pd.to_datetime(dt[1]+' '+ dt[3]+ ' '+ dt[5])


In [106]:
df= df.replace({'\'': ''}, regex = True)

 

In [107]:

df["CloudCover"]= df["CloudCover"].astype(float, errors= 'ignore')
df["Max.Dew.PointF"]= df["Max.Dew.PointF"].astype(float, errors= 'ignore')

df["Max.Humidity"]= df["Max.Humidity"].astype(float, errors= 'ignore')
df["Max.Sea.Level.PressureIn"]= df["Max.Sea.Level.PressureIn"].astype(float, errors= 'ignore')
df["Max.TemperatureF"]= df["Max.TemperatureF"].astype(float, errors= 'ignore')
df["Max.VisibilityMiles"]= df["Max.VisibilityMiles"].astype(float, errors= 'ignore')
df["Max.Wind.SpeedMPH"]= df["Max.Wind.SpeedMPH"].astype(float, errors= 'ignore')
df["Mean.Humidity"]= df["CloudCover"].astype(float, errors= 'ignore')
df["Mean.VisibilityMiles"]= df["Mean.VisibilityMiles"].astype(float, errors= 'ignore')
df["MeanDew.PointF"]= df["MeanDew.PointF"].astype(float, errors= 'ignore')
df["Min.Humidity"]= df["Min.Humidity"].astype(float, errors= 'ignore')
df["Mean.Sea.Level.PressureIn"]= df["Mean.Sea.Level.PressureIn"].astype(float, errors= 'ignore')
df["Mean.TemperatureF"]= df["Mean.TemperatureF"].astype(float, errors= 'ignore')
df["Mean.Wind.SpeedMPH"]= df["Mean.Wind.SpeedMPH"].astype(float, errors= 'ignore')
df["Min.DewpointF"]= df["Min.DewpointF"].astype(float, errors= 'ignore')
df["Min.Sea.Level.PressureIn"]= df["Min.Sea.Level.PressureIn"].astype(float, errors= 'ignore')
df["Min.TemperatureF"]= df["Min.TemperatureF"].astype(float, errors= 'ignore')
df["PrecipitationIn"]= df["PrecipitationIn"].astype(float, errors= 'ignore')
df["WindDirDegrees"]= df["WindDirDegrees"].astype(float, errors= 'ignore')
df["Min.VisibilityMiles"]= df["Min.VisibilityMiles"].astype(float, errors= 'ignore')
df






,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,6.0,Rain,46.0,29,74.0,30.45,64.0,10.0,22.0,6.0,...,13.0,40.0,26.0,52.0,30.01,39.0,10.0,0.01,268.0,2014-12-01
1,7.0,Rain-Snow,40.0,29,92.0,30.71,42.0,10.0,24.0,7.0,...,15.0,27.0,17.0,51.0,30.40,33.0,2.0,0.10,62.0,2014-12-02
2,8.0,Rain,49.0,38,100.0,30.40,51.0,10.0,29.0,8.0,...,12.0,42.0,24.0,57.0,29.87,37.0,1.0,0.44,254.0,2014-12-03
3,3.0,NaN,24.0,33,69.0,30.56,43.0,10.0,25.0,3.0,...,12.0,21.0,13.0,39.0,30.09,30.0,10.0,0.00,292.0,2014-12-04
4,5.0,Rain,37.0,26,85.0,30.68,42.0,10.0,22.0,5.0,...,10.0,25.0,12.0,47.0,30.45,26.0,5.0,0.11,61.0,2014-12-05
5,8.0,Rain,45.0,25,100.0,30.42,45.0,10.0,22.0,8.0,...,8.0,40.0,36.0,85.0,30.16,38.0,0.0,1.09,313.0,2014-12-06
6,6.0,Rain,36.0,32,92.0,30.69,38.0,10.0,25.0,6.0,...,15.0,20.0,-3.0,29.0,30.24,21.0,5.0,0.13,350.0,2014-12-07
7,8.0,Snow,28.0,28,92.0,30.77,29.0,10.0,21.0,8.0,...,13.0,16.0,3.0,47.0,30.51,18.0,2.0,0.03,354.0,2014-12-08
8,8.0,Rain,49.0,52,100.0,30.51,49.0,10.0,38.0,8.0,...,20.0,41.0,28.0,86.0,29.49,29.0,1.0,2.90,38.0,2014-12-09
9,8.0,Rain,45.0,29,100.0,29.58,48.0,10.0,23.0,8.0,...,13.0,39.0,37.0,89.0,29.43,38.0,1.0,0.28,357.0,2014-12-10


## Distributions + Removing outliers + Imputing missing values

-----------------------------------------------------------------

- Now, you can start the process of data cleaning and exploration as these two things go hand-in-hand. Look for missing values and outliers. You can use different methods to find these problem points using visual methods like plots or analytical methods like summary statistics. 
- Look at what plots can be used to find outliers. Which columns have outliers? What are those values? What did you to process them?
- Which columns have missing values? Find all the columns of possible missing values and impute them using the correct method.
- Check for positive and negative values for all columns. Is there any value which is not allowed to be in a column? What can be done to correct these?

In [108]:




        
        


df["Max.Gust.SpeedMPH"]= df["Max.Gust.SpeedMPH"].astype(float)

m0= df['Max.Gust.SpeedMPH']

        
median= sts.median(m0)



df['Max.Gust.SpeedMPH']= df['Max.Gust.SpeedMPH'].replace(to_replace= 0 , value=median)
df




,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,6.0,Rain,46.0,29.0,74.0,30.45,64.0,10.0,22.0,6.0,...,13.0,40.0,26.0,52.0,30.01,39.0,10.0,0.01,268.0,2014-12-01
1,7.0,Rain-Snow,40.0,29.0,92.0,30.71,42.0,10.0,24.0,7.0,...,15.0,27.0,17.0,51.0,30.40,33.0,2.0,0.10,62.0,2014-12-02
2,8.0,Rain,49.0,38.0,100.0,30.40,51.0,10.0,29.0,8.0,...,12.0,42.0,24.0,57.0,29.87,37.0,1.0,0.44,254.0,2014-12-03
3,3.0,NaN,24.0,33.0,69.0,30.56,43.0,10.0,25.0,3.0,...,12.0,21.0,13.0,39.0,30.09,30.0,10.0,0.00,292.0,2014-12-04
4,5.0,Rain,37.0,26.0,85.0,30.68,42.0,10.0,22.0,5.0,...,10.0,25.0,12.0,47.0,30.45,26.0,5.0,0.11,61.0,2014-12-05
5,8.0,Rain,45.0,25.0,100.0,30.42,45.0,10.0,22.0,8.0,...,8.0,40.0,36.0,85.0,30.16,38.0,0.0,1.09,313.0,2014-12-06
6,6.0,Rain,36.0,32.0,92.0,30.69,38.0,10.0,25.0,6.0,...,15.0,20.0,-3.0,29.0,30.24,21.0,5.0,0.13,350.0,2014-12-07
7,8.0,Snow,28.0,28.0,92.0,30.77,29.0,10.0,21.0,8.0,...,13.0,16.0,3.0,47.0,30.51,18.0,2.0,0.03,354.0,2014-12-08
8,8.0,Rain,49.0,52.0,100.0,30.51,49.0,10.0,38.0,8.0,...,20.0,41.0,28.0,86.0,29.49,29.0,1.0,2.90,38.0,2014-12-09
9,8.0,Rain,45.0,29.0,100.0,29.58,48.0,10.0,23.0,8.0,...,13.0,39.0,37.0,89.0,29.43,38.0,1.0,0.28,357.0,2014-12-10


In [109]:
df['Events']= df['Events'].replace(to_replace= np.NaN, value='None')
df

,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,6.0,Rain,46.0,29.0,74.0,30.45,64.0,10.0,22.0,6.0,...,13.0,40.0,26.0,52.0,30.01,39.0,10.0,0.01,268.0,2014-12-01
1,7.0,Rain-Snow,40.0,29.0,92.0,30.71,42.0,10.0,24.0,7.0,...,15.0,27.0,17.0,51.0,30.40,33.0,2.0,0.10,62.0,2014-12-02
2,8.0,Rain,49.0,38.0,100.0,30.40,51.0,10.0,29.0,8.0,...,12.0,42.0,24.0,57.0,29.87,37.0,1.0,0.44,254.0,2014-12-03
3,3.0,None,24.0,33.0,69.0,30.56,43.0,10.0,25.0,3.0,...,12.0,21.0,13.0,39.0,30.09,30.0,10.0,0.00,292.0,2014-12-04
4,5.0,Rain,37.0,26.0,85.0,30.68,42.0,10.0,22.0,5.0,...,10.0,25.0,12.0,47.0,30.45,26.0,5.0,0.11,61.0,2014-12-05
5,8.0,Rain,45.0,25.0,100.0,30.42,45.0,10.0,22.0,8.0,...,8.0,40.0,36.0,85.0,30.16,38.0,0.0,1.09,313.0,2014-12-06
6,6.0,Rain,36.0,32.0,92.0,30.69,38.0,10.0,25.0,6.0,...,15.0,20.0,-3.0,29.0,30.24,21.0,5.0,0.13,350.0,2014-12-07
7,8.0,Snow,28.0,28.0,92.0,30.77,29.0,10.0,21.0,8.0,...,13.0,16.0,3.0,47.0,30.51,18.0,2.0,0.03,354.0,2014-12-08
8,8.0,Rain,49.0,52.0,100.0,30.51,49.0,10.0,38.0,8.0,...,20.0,41.0,28.0,86.0,29.49,29.0,1.0,2.90,38.0,2014-12-09
9,8.0,Rain,45.0,29.0,100.0,29.58,48.0,10.0,23.0,8.0,...,13.0,39.0,37.0,89.0,29.43,38.0,1.0,0.28,357.0,2014-12-10


Your dataset should look like `weather_clean.pkl` file at the end. Use it as a refrence.

Let's take a look at our final cleaned dataset before saving it for analysis. We will save it as a 'pickle', a binary file format which will preserve out column types. Text-based file formats like csv and json do not preserve column data types. 

In [110]:

df['PrecipitationIn']= df['PrecipitationIn'].replace(to_replace= 'T', value='0')

df["PrecipitationIn"]= df["PrecipitationIn"].astype(float, errors= 'ignore')



df
        

,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees,Date
0,6.0,Rain,46.0,29.0,74.0,30.45,64.0,10.0,22.0,6.0,...,13.0,40.0,26.0,52.0,30.01,39.0,10.0,0.01,268.0,2014-12-01
1,7.0,Rain-Snow,40.0,29.0,92.0,30.71,42.0,10.0,24.0,7.0,...,15.0,27.0,17.0,51.0,30.40,33.0,2.0,0.10,62.0,2014-12-02
2,8.0,Rain,49.0,38.0,100.0,30.40,51.0,10.0,29.0,8.0,...,12.0,42.0,24.0,57.0,29.87,37.0,1.0,0.44,254.0,2014-12-03
3,3.0,None,24.0,33.0,69.0,30.56,43.0,10.0,25.0,3.0,...,12.0,21.0,13.0,39.0,30.09,30.0,10.0,0.00,292.0,2014-12-04
4,5.0,Rain,37.0,26.0,85.0,30.68,42.0,10.0,22.0,5.0,...,10.0,25.0,12.0,47.0,30.45,26.0,5.0,0.11,61.0,2014-12-05
5,8.0,Rain,45.0,25.0,100.0,30.42,45.0,10.0,22.0,8.0,...,8.0,40.0,36.0,85.0,30.16,38.0,0.0,1.09,313.0,2014-12-06
6,6.0,Rain,36.0,32.0,92.0,30.69,38.0,10.0,25.0,6.0,...,15.0,20.0,-3.0,29.0,30.24,21.0,5.0,0.13,350.0,2014-12-07
7,8.0,Snow,28.0,28.0,92.0,30.77,29.0,10.0,21.0,8.0,...,13.0,16.0,3.0,47.0,30.51,18.0,2.0,0.03,354.0,2014-12-08
8,8.0,Rain,49.0,52.0,100.0,30.51,49.0,10.0,38.0,8.0,...,20.0,41.0,28.0,86.0,29.49,29.0,1.0,2.90,38.0,2014-12-09
9,8.0,Rain,45.0,29.0,100.0,29.58,48.0,10.0,23.0,8.0,...,13.0,39.0,37.0,89.0,29.43,38.0,1.0,0.28,357.0,2014-12-10


In [111]:
m0=[]
m0= df['Mean.VisibilityMiles']

for i in range(len(m0)):
    if(m0[i]<0):
         m0[i]=0
    


        

        

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [112]:
ds= df.loc[ [], 'Max.Dew.PointF':'Min.TemperatureF']
for column in ds:

    m0= df[column]
    obj=[]
    obj=m0
    median= sts.median(m0)
    Q1 = obj.quantile(0.25)
    Q3 = obj.quantile(0.75)
    IQR = Q3 - Q1

    for i in range(len(obj)):
        if  obj[i] < (Q1 - 1.5 * IQR) or obj[i] > (Q3 + 1.5 * IQR):
           
            (obj[i])= median

     
       


        


    
    
    


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [113]:
#save the cleaned dataset
#df.to_pickle('weather_clean.pkl')

In [115]:
df.to_pickle('weather_clean.pkl')